# 🚀 HVAC AI — YOLO11 Inference Server
**Quick Start for YOLO11 Deployment**

---

## 📋 Simple 4-Step Deployment

This is the **minimal** version for quick deployment. For production use with validation, monitoring, and testing, see:
- `hvac-inference_yolo_enhanced.ipynb` - Full production-ready version

### Prerequisites
1. Set Colab runtime to **GPU** (Runtime → Change runtime type → GPU)
2. Have your trained YOLO11 model (`.pt` file) in Google Drive
3. Get ngrok token from [ngrok.com](https://ngrok.com/)

### Steps
1. **Setup** - Install dependencies
2. **Mount Drive** - Access your model
3. **Configure** - Set paths and tokens
4. **Launch** - Start the API server

---

In [ ]:
# 1. Setup Environment
import sys
import torch

print("📦 Cloning repository...")
!git clone https://github.com/elliotttmiller/hvac-ai.git 2>/dev/null || echo "Repository exists"
%cd hvac-ai

print("\n📚 Installing dependencies...")
!pip install -q ultralytics>=8.0.0 fastapi>=0.115.0 uvicorn[standard]>=0.34.0
!pip install -q python-multipart>=0.0.9 pyngrok>=7.0.0 python-dotenv>=1.0.0

print("\n🔍 Validating environment...")
print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB)")
else:
    print("⚠️  WARNING: No GPU detected! Set Runtime → Change runtime type → GPU")

print('\n✅ Environment Ready')

In [ ]:
# 2. Mount Drive (To access your best.pt)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%writefile .env
# 3. CONFIGURATION - UPDATE THESE VALUES
# Get ngrok token from: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTHTOKEN="YOUR_NGROK_TOKEN_HERE"
PORT=8000

# Point this to your trained YOLO11 model in Google Drive
# Example path format: /content/drive/MyDrive/your_project/weights/best.pt
MODEL_PATH="/content/drive/MyDrive/hvac_detection_project/runs/segment/hvac_yolo11_segmentation_sota/weights/best.pt"

# Optional: Adjust inference settings
# DEFAULT_CONF_THRESHOLD=0.50
# DEFAULT_IOU_THRESHOLD=0.45

In [ ]:
# 4. Launch Server
import os
from pyngrok import ngrok
from dotenv import load_dotenv

print("🚀 Launching API Server")
print("="*70)

load_dotenv()

# Validate configuration
model_path = os.getenv('MODEL_PATH')
ngrok_token = os.getenv('NGROK_AUTHTOKEN')

if not model_path or not os.path.exists(model_path):
    print("❌ ERROR: MODEL_PATH not found. Check your .env configuration above.")
    raise FileNotFoundError(f"Model not found: {model_path}")

print(f"✅ Model found: {model_path}")

# Setup ngrok tunnel
if ngrok_token and ngrok_token != "YOUR_NGROK_TOKEN_HERE":
    print("\n🌐 Setting up ngrok tunnel...")
    ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(8000)
    print(f"\n✅ API LIVE!")
    print(f"   Public URL: {public_url.public_url}")
    print(f"   API Docs: {public_url.public_url}/docs")
    print(f"   Health: {public_url.public_url}/health")
else:
    print("\n⚠️  No ngrok token - server will be local only")
    print("   Local URL: http://localhost:8000")

print("\n" + "="*70)
print("🎬 Starting server (Press STOP button to shutdown)...")
print("="*70 + "\n")

%cd python-services
!uvicorn hvac_analysis_service:app --host 0.0.0.0 --port 8000 --reload